# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [3]:
import numpy as np
import pandas as pd
from scipy import stats
import math
import xlsxwriter
import requests

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'



## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [5]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data


{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'GAEALKNM TADCAOSEBL /SGLN)S(TRE Q',
 'calculationPrice': 'iexlasttrade',
 'open': None,
 'openTime': None,
 'openSource': 'aofifilc',
 'close': None,
 'closeTime': None,
 'closeSource': 'foiiaclf',
 'high': None,
 'highTime': None,
 'highSource': None,
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 136.18,
 'latestSource': 'IEX Last Trade',
 'latestTime': 'April 21, 2021',
 'latestUpdate': 1625404089599,
 'latestVolume': None,
 'iexRealtimePrice': 137.01,
 'iexRealtimeSize': 103,
 'iexLastUpdated': 1632004423556,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 137.33,
 'previousVolume': 96583636,
 'change': 0.38,
 'changePercent': 0.00299,
 'volume': None,
 'iexMarketPercent': 0.016600495671909053,
 'iexVolume

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [6]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio # be careful about what the Ratio IS



37.68

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [7]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url= f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ],
            index = my_columns
        ), ignore_index = True
        )
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,139.04,52.29,N/A
1,AAL,22.00,-1.17,N/A
2,AAP,209.34,28,N/A
3,AAPL,138.35,37.18,N/A
4,ABBV,111.80,41.31,N/A
...,...,...,...,...
500,YUM,118.53,40.05,N/A
501,ZBH,177.06,-272.34,N/A
502,ZBRA,498.53,53.74,N/A
503,ZION,55.48,8.78,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [9]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe.drop('index',axis=1, inplace = True)


In [10]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,646.220,5.05,N/A
1,ALL,129.400,7.35,N/A
2,UNM,29.970,7.58,N/A
3,EBAY,64.183,8.05,N/A
4,AFL,55.800,8.21,N/A
5,ZION,55.480,8.78,N/A
6,COO,412.750,9.2,N/A
7,KIM,21.140,9.36,N/A
8,CE,159.340,9.66,N/A
9,CTL,11.000,9.82,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [11]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [12]:

portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [13]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row,'Price'])

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,646.220,5.05,30
1,ALL,129.400,7.35,154
2,UNM,29.970,7.58,667
3,EBAY,64.183,8.05,311
4,AFL,55.800,8.21,358
5,ZION,55.480,8.78,360
6,COO,412.750,9.2,48
7,KIM,21.140,9.36,946
8,CE,159.340,9.66,125
9,CTL,11.000,9.82,1818


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [16]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [17]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

In [18]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.14,N/A,53.6,N/A,8.75,N/A,7.68,N/A,33.436672,N/A,14.333927,N/A,N/A
1,AAL,21.00,N/A,-1.14,N/A,-2.11,N/A,0.841,N/A,-4.439409,N/A,2.274120,N/A,N/A
2,AAP,205.01,N/A,29.26,N/A,3.69,N/A,1.28,N/A,12.533828,N/A,2.832879,N/A,N/A
3,AAPL,139.15,N/A,37.39,N/A,34.37,N/A,7.86,N/A,27.222835,N/A,19.860749,N/A,N/A
4,ABBV,113.10,N/A,41.4,N/A,15.15,N/A,4.31,N/A,14.274893,N/A,8.794709,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,120.96,N/A,40.91,N/A,-4.6,N/A,6.4,N/A,25.241611,N/A,11.068723,N/A,N/A
501,ZBH,181.05,N/A,-267.7,N/A,2.98,N/A,5.1,N/A,31.225370,N/A,8.852591,N/A,N/A
502,ZBRA,512.12,N/A,54.34,N/A,13.35,N/A,6.38,N/A,37.533613,N/A,14.386100,N/A,N/A
503,ZION,55.58,N/A,8.55,N/A,1.25,N/A,2.69,N/A,7.029251,N/A,3.025709,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [24]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [23]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [20]:
rv_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'Price-to-Earnings Ratio',
       'PE Percentile', 'Price-to-Book Ratio', 'PB Percentile',
       'Price-to-Sales Ratio', 'PS Percentile', 'EV/EBITDA',
       'EV/EBITDA Percentile', 'EV/GP', 'EV/GP Percentile', 'RV Score'],
      dtype='object')

## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [28]:
from scipy.stats import percentileofscore as score

metrics = {'Price-to-Earnings Ratio':'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile'}
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score( rv_dataframe[metric], rv_dataframe.loc[row,metric])
        
rv_dataframe


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.14,N/A,53.60,83.8614,8.75,74.2574,7.680,79.4059,33.436672,86.9307,14.333927,78.0198,N/A
1,AAL,21.00,N/A,-1.14,15.4455,-2.11,4.55446,0.841,6.33663,-4.439409,3.9604,2.274120,7.92079,N/A
2,AAP,205.01,N/A,29.26,51.8812,3.69,45.0495,1.280,13.8614,12.533828,27.5248,2.832879,11.2871,N/A
3,AAPL,139.15,N/A,37.39,70.099,34.37,95.4455,7.860,80.297,27.222835,78.6139,19.860749,91.0891,N/A
4,ABBV,113.10,N/A,41.40,74.8515,15.15,87.5248,4.310,56.8317,14.274893,36.2376,8.794709,50.495,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,120.96,N/A,40.91,73.8614,-4.60,4.35644,6.400,74.4554,25.241611,76.4356,11.068723,67.1287,N/A
501,ZBH,181.05,N/A,-267.70,0.990099,2.98,37.2277,5.100,63.1683,31.225370,85.7426,8.852591,50.8911,N/A
502,ZBRA,512.12,N/A,54.34,84.3564,13.35,85.5446,6.380,74.2574,37.533613,88.3168,14.386100,78.6139,N/A
503,ZION,55.58,N/A,8.55,17.0297,1.25,9.90099,2.690,37.4257,7.029251,6.53465,3.025709,12.2772,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [32]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row,metrics[metric]])
        
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
rv_dataframe


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.14,N/A,53.60,83.8614,8.75,74.2574,7.680,79.4059,33.436672,86.9307,14.333927,78.0198,80.495
1,AAL,21.00,N/A,-1.14,15.4455,-2.11,4.55446,0.841,6.33663,-4.439409,3.9604,2.274120,7.92079,7.64356
2,AAP,205.01,N/A,29.26,51.8812,3.69,45.0495,1.280,13.8614,12.533828,27.5248,2.832879,11.2871,29.9208
3,AAPL,139.15,N/A,37.39,70.099,34.37,95.4455,7.860,80.297,27.222835,78.6139,19.860749,91.0891,83.1089
4,ABBV,113.10,N/A,41.40,74.8515,15.15,87.5248,4.310,56.8317,14.274893,36.2376,8.794709,50.495,61.1881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,120.96,N/A,40.91,73.8614,-4.60,4.35644,6.400,74.4554,25.241611,76.4356,11.068723,67.1287,59.2475
501,ZBH,181.05,N/A,-267.70,0.990099,2.98,37.2277,5.100,63.1683,31.225370,85.7426,8.852591,50.8911,47.604
502,ZBRA,512.12,N/A,54.34,84.3564,13.35,85.5446,6.380,74.2574,37.533613,88.3168,14.386100,78.6139,82.2178
503,ZION,55.58,N/A,8.55,17.0297,1.25,9.90099,2.690,37.4257,7.029251,6.53465,3.025709,12.2772,16.6337


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [36]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,29.730,N/A,7.49,16.4356,0.5692,4.75248,0.4732,2.17822,3.707982,4.75248,0.437798,0.39604,5.70297
1,MCK,198.300,N/A,-8.06,10.8911,-66.0700,0.39604,0.1356,0.594059,7.388476,6.93069,2.744175,10.8911,5.94059
2,FTI,7.430,N/A,-1.02,15.6436,0.7932,5.74257,0.2565,0.990099,2.085225,4.15842,1.271499,3.16832,5.94059
3,AIG,47.540,N/A,-7.00,11.3861,0.6244,5.34653,0.9567,8.11881,7.220102,6.73267,0.915608,1.9802,6.71287
4,AAL,21.000,N/A,-1.14,15.4455,-2.1100,4.55446,0.8410,6.33663,-4.439409,3.9604,2.274120,7.92079,7.64356
5,MET,63.270,N/A,11.50,19.604,0.7690,5.54455,0.8558,6.73267,5.532635,5.14851,0.827863,1.58416,7.72277
6,ALL,124.200,N/A,7.31,16.2376,1.4000,13.3663,0.8393,5.94059,2.686042,4.55446,0.817299,1.18812,8.25743
7,F,12.140,N/A,-36.56,5.34653,1.6100,17.7228,0.3900,1.78218,6.361740,5.74257,2.865905,11.4851,8.41584
8,DXC,32.630,N/A,-2.92,13.6634,1.4400,14.4554,0.4605,1.9802,6.044770,5.54455,2.722293,10.495,9.22772
9,ABC,122.420,N/A,-7.90,11.0891,-36.4100,1.18812,0.1315,0.39604,9.236167,13.2673,4.494672,21.5842,9.50495


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [37]:
portfolio_input()

Enter the value of your portfolio:1000000


In [38]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

C:\Users\robzh\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,29.730,672,7.49,16.4356,0.5692,4.75248,0.4732,2.17822,3.707982,4.75248,0.437798,0.39604,5.70297
1,MCK,198.300,100,-8.06,10.8911,-66.0700,0.39604,0.1356,0.594059,7.388476,6.93069,2.744175,10.8911,5.94059
2,FTI,7.430,2691,-1.02,15.6436,0.7932,5.74257,0.2565,0.990099,2.085225,4.15842,1.271499,3.16832,5.94059
3,AIG,47.540,420,-7.00,11.3861,0.6244,5.34653,0.9567,8.11881,7.220102,6.73267,0.915608,1.9802,6.71287
4,AAL,21.000,952,-1.14,15.4455,-2.1100,4.55446,0.8410,6.33663,-4.439409,3.9604,2.274120,7.92079,7.64356
5,MET,63.270,316,11.50,19.604,0.7690,5.54455,0.8558,6.73267,5.532635,5.14851,0.827863,1.58416,7.72277
6,ALL,124.200,161,7.31,16.2376,1.4000,13.3663,0.8393,5.94059,2.686042,4.55446,0.817299,1.18812,8.25743
7,F,12.140,1647,-36.56,5.34653,1.6100,17.7228,0.3900,1.78218,6.361740,5.74257,2.865905,11.4851,8.41584
8,DXC,32.630,612,-2.92,13.6634,1.4400,14.4554,0.4605,1.9802,6.044770,5.54455,2.722293,10.495,9.22772
9,ABC,122.420,163,-7.90,11.0891,-36.4100,1.18812,0.1315,0.39604,9.236167,13.2673,4.494672,21.5842,9.50495


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [39]:
writer = pd.ExcelWriter('Value_strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [40]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [41]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [42]:
writer.save()